In [6]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [7]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [8]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

# 1. DQN Agent

In [9]:
from agent import PPOAgent
from agent.model import ConvActor, ConvCritic
from agent.configuration import PPOConfig
from agent.exploration import LinearExploration

In [10]:
actor_model = ConvActor(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=16,
    action_dim=49
)
critic_model = ConvCritic(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=16
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = PPOConfig(
    actor_model=actor_model,
    critic_model=critic_model,
    ckpt_path='./PPO_gobang_n7',
)
agent = PPOAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/ppo_gobang_n7')

2023-05-05 06:05:40.479 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 16.90496964752674, reward sum: -42
2023-05-05 06:05:46.895 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 15/50
2023-05-05 06:05:47.443 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 12.830105602741241, reward sum: -31
2023-05-05 06:05:48.244 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 8.873746279627085, reward sum: -38
2023-05-05 06:05:48.976 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 9.985255530104041, reward sum: -35
2023-05-05 06:05:49.491 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 6.093772932887077, reward sum: -27
2023-05-05 06:05:50.407 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 6.997288938611746, reward sum: -33
2023-05-05 06:05:51.183 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 4.59914243966341, reward sum: -33
2023-05-05 06:05:51.901 | INFO     | __main__:do_exp:10

In [11]:
actor_model = ConvActor(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=16,
    action_dim=121
)
critic_model = ConvCritic(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=16
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = PPOConfig(
    actor_model=actor_model,
    critic_model=critic_model,
    ckpt_path='./PPO_gobang_n11',
)
agent = PPOAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/ppo_gobang_n11')

2023-05-05 06:28:05.410 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 8.621211489662528, reward sum: -85
2023-05-05 06:28:18.396 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 24/50
2023-05-05 06:28:20.142 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 5.156345237977803, reward sum: -75
2023-05-05 06:28:21.330 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 3.7926857881247997, reward sum: -59
2023-05-05 06:28:22.981 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 2.9888288942165673, reward sum: -89
2023-05-05 06:28:24.761 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 3.0462337390054017, reward sum: -80
2023-05-05 06:28:25.932 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.9370777613172928, reward sum: -58
2023-05-05 06:28:27.355 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 1.1323030247197796, reward sum: -49
2023-05-05 06:28:28.611 | INFO     | __main__:do_e